# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [23]:
%pip install azureml-train-automl
%pip install azureml-widgets
%pip install azureml-pipeline
%pip install azureml-pipeline-steps
%pip install kaggle


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [24]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.webservice.aci import AciWebservice
from azureml.data import TabularDataset
from azureml.exceptions import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep, PythonScriptStep



In [25]:
%kaggle datasets download -d rezaunderfit/instagram-fake-and-real-accounts-dataset

UsageError: Line magic function `%kaggle` not found.


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [26]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-instagram-users-train'
project_folder = './automl-pipeline-project'
experiment=Experiment(ws, experiment_name)


In [27]:
import pandas as pd

key = "instagram-fake-real-accounts"
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
else:
    dataset = pd.read_csv("./kaggle_data_ig_accounts/final-v1.csv")
    dataset = TabularDataset.Tabular.register_pandas_dataframe(dataframe=dataset, target=ws.get_default_datastore(), name=key, description="Kaggle dataset of Instagram accounts both real and fake.")

In [32]:
compute_cluster_name = "my-udacity-compute"

try:
    compute_cluster = ComputeTarget(ws, compute_cluster_name)
    print("Found an existing cluster, using it")
except ComputeTargetException as ex:
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_cluster_config)
    
compute_cluster.wait_for_completion(show_output=True, timeout_in_minutes=10)

Found an existing cluster, using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Configuration Settings Rationale

- task: Since the dataset schema contains a label column `is_fake` with Boolean values, a classification model is the obvious choice for training. 
- `max_concurrent_iterations`: Determined by the size of the largest VM we are permitted to use for compute in the Udacity environment
- `primary_metric`: To get a baseline model for evaluation, I chose to optimize accuracy for the AutoML run. However, that metric "may not optimize as well for datasets that are small, have very large class skew (class imbalance), or when the expected metric value is very close to 0.0 or 1.0" (Microsoft Learn, "Set up AutoML training with Python º Metrics for classification scenarios")[https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-configure-auto-train-v1#metrics-for-classification-scenarios]. My dataset meets at least one and possibly two of these criteria (less than 800 records, and ~88% of them are labeled as fake). As such, after obtaining a baseline model and its performance metrics, I intend to run a second AutoML training job with weighted AUC as the primary metric, and then compare its performance to the first job's output.
- `featurization`: Intentionally left as the default value `auto`. 

In [56]:
automl_settings = {
    "max_concurrent_iterations": 3,
    "experiment_timeout_minutes": 25,
    "primary_metric": "AUC_weighted", 
    "n_cross_validations": 5
}

automl_config = AutoMLConfig(task="classification", 
                             path = project_folder,
                             compute_target=compute_cluster,
                             training_data = dataset,
                             label_column_name = "is_fake",
                             debug_log = "automl_run_errors.log",
                             **automl_settings)


In [35]:
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.core import PipelineParameter

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics')
                           )
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model')
                           )

automl_step = AutoMLStep(
    name = 'automl_module', 
    automl_config = automl_config,
    outputs = [metrics_data, model_data],
    allow_reuse = True
)



In [57]:
#automl_pipeline = Pipeline(workspace=ws, description="An automated ML pipeline for training a classifier to detect fake IG accounts", steps = [automl_step])
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-instagram-users-train,AutoML_d473aa69-8dd1-447b-b771-68cceb5cda76,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?


In [58]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model



In [59]:
import pickle

best_run = automl_run.get_best_child()
print(best_run.properties)

{'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-instagram-users-train\',\'compute_target\':\'my-udacity-compute\',\'subscription_id\':\'610d6e37-4747-4a20-80eb-3aad70a55f43\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_d473aa69-8dd1-447b-b771-68cceb5cda76_42","experiment_name":"udacity-instagram-users-train","workspace_name":"quick-starts-ws-217182","subscription_id":"610d6e37-4747-4a20-80eb-3aad70a55f43","resource_group_name":"aml-quickstarts-217182"}}]}', 'training_percent': '100', 'predicted_cost': None, 'iteration': '42', '_aml_system_scenario_identification': 'Remote

In [60]:
best_run.download_file("outputs/scoring_file_v_1_0_0.py", "ig_classifier_service/score.py")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [61]:
from azureml.core import Model

#Note: have been suddenly having much difficulty in recent days with azureml methods not finding the model & metric files from finished runs,
#even after having just downloaded them with e.g. best_model_output.download(). Due to this, I must use the register method on the Model
#class to register the model, though I would have greatly preferred to do it through the pipeline_run object itself.  
# registered_model = Model.register(workspace=ws, 
#                                     model_path=best_model_output._path_on_datastore,
#                                     model_name="best_model"
# )

model_name = best_run.properties["model_name"]
description_for_model = "AutoML classifier model for predicting whether an Instagram account is inauthentic (in the Meta Community Standards sense)."
registered_model = automl_run.register_model(
    model_name=model_name,
    description=description_for_model
)


In [50]:
import os

folder_name = "ig_classifier_service"
os.makedirs(f"./{folder_name}", exist_ok=True)

In [62]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script = "score.py",
                                    runtime = "python",
                                    source_directory=folder_name)

deploy_config = AciWebservice.deploy_configuration(cpu_cores=1)

In [64]:
automl_service = Model.deploy(workspace=ws, name="ig-service",
                                models=[registered_model],
                                inference_config=inference_config,
                                deployment_config=deploy_config)
automl_service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-02 02:56:26+00:00 Creating Container Registry if not exists.
2022-12-02 02:56:26+00:00 Registering the environment.
2022-12-02 02:56:27+00:00 Use the existing image.
2022-12-02 02:56:28+00:00 Generating deployment configuration.
2022-12-02 02:56:28+00:00 Submitting deployment to compute.
2022-12-02 02:56:32+00:00 Checking the status of deployment ig-service..
2022-12-02 02:59:17+00:00 Checking the status of inference endpoint ig-service.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: eac7a66d-467c-4492-87f2-563a99b3302a
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: ig-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 859a61f3b0124a4cad3910ba11916195.azurecr.io/azureml/azureml_ede5d179cfd1af6f52d27ff88a160155 locally. Please refer to https:/

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: eac7a66d-467c-4492-87f2-563a99b3302a
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: ig-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 859a61f3b0124a4cad3910ba11916195.azurecr.io/azureml/azureml_ede5d179cfd1af6f52d27ff88a160155 locally. Please refer

TODO: In the cell below, send a request to the web service you deployed to test it.

In [65]:
print(automl_service.get_logs())

None


TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
